In [ ]:
#!pip install opencv-python numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 20.1 MB/s eta 0:00:0000:0100:01
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)


In [1]:
import cv2
import numpy as np
import math

In [2]:
ref_points_1 = [] # Left image points
ref_points_2 = [] # Right image points
test_mode = False

# ==========================================
# PART 1: FUNDAMENTAL MATRIX & EPIPOLES
# ==========================================

In [3]:
def normalize_points(pts, width, height):
    """
    Normalizes points to improve 8-point algorithm stability.
    Translate centroid to origin and scale so average distance is sqrt(2).
    """
    pts = np.array(pts)
    centroid = np.mean(pts, axis=0)
    
    # Shift origin to centroid
    shifted_pts = pts - centroid
    
    # Calculate average distance from origin
    mean_dist = np.mean(np.sqrt(np.sum(shifted_pts**2, axis=1)))
    
    # Scale factor
    scale = np.sqrt(2) / mean_dist
    
    # Construct transformation matrix T
    T = np.array([
        [scale, 0, -scale * centroid[0]],
        [0, scale, -scale * centroid[1]],
        [0, 0, 1]
    ])
    
    # Apply T to points (convert to homogeneous coords first)
    pts_h = np.column_stack((pts, np.ones(len(pts))))
    pts_norm = (T @ pts_h.T).T
    
    return pts_norm[:, :2], T

In [4]:
def compute_fundamental_matrix(pts1, pts2):
    """
    Computes F using the normalized 8-point algorithm.
    Reference: Stereo Vision Slides, Page 32.
    """
    h, w = 1000, 1000 # Arbitrary for normalization, just need image dims
    
    # 1. Normalize points
    pts1_norm, T1 = normalize_points(pts1, w, h)
    pts2_norm, T2 = normalize_points(pts2, w, h)
    
    # 2. Build Constraint Matrix A
    # Equation: p2' * F * p1 = 0 -> [u'u, u'v, u', v'u, v'v, v', u, v, 1] * f = 0
    A = []
    for i in range(len(pts1)):
        u, v = pts1_norm[i]
        u_p, v_p = pts2_norm[i]
        A.append([u_p*u, u_p*v, u_p, v_p*u, v_p*v, v_p, u, v, 1])
    A = np.array(A)
    
    # 3. SVD of A to find F
    U, S, Vt = np.linalg.svd(A)
    F_prime = Vt[-1].reshape(3, 3)
    
    # 4. Enforce Rank 2 Constraint (Singularity constraint)
    # Reference: Stereo Vision Slides, Page 32 ("Set smallest singular value to 0")
    Uf, Sf, Vtf = np.linalg.svd(F_prime)
    Sf[2] = 0 # Zero out smallest singular value
    F_rank2 = Uf @ np.diag(Sf) @ Vtf
    
    # 5. De-normalize: F = T2' * F_rank2 * T1
    F = T2.T @ F_rank2 @ T1
    
    # Normalize F so last element is 1 (standard convention)
    return F / F[2, 2]

In [5]:
def compute_epipoles(F):
    """
    Computes epipoles using SVD.
    Reference: Stereo Vision Slides, Page 33.
    """
    # Epipole e1 (left) is null space of F: F * e1 = 0
    U, S, Vt = np.linalg.svd(F)
    e1 = Vt[-1]
    e1 = e1 / e1[2] # Normalize
    
    # Epipole e2 (right) is null space of F.T: F.T * e2 = 0
    U, S, Vt = np.linalg.svd(F.T)
    e2 = Vt[-1]
    e2 = e2 / e2[2] # Normalize
    
    return e1, e2

In [6]:
def compute_epipolar_line(pt, F, which_image):
    """
    Computes the line equation ax + by + c = 0.
    If pt is in image 1, line is in image 2: l' = F * p
    If pt is in image 2, line is in image 1: l = F.T * p'
    """
    pt_h = np.array([pt[0], pt[1], 1])
    
    if which_image == 1: # Point in Img1, line in Img2
        line = F @ pt_h
    else: # Point in Img2, line in Img1
        line = F.T @ pt_h
        
    return line

In [7]:
def calculate_distance_to_line(pt, line):
    """ Calculates geometric distance from point (x,y) to line ax+by+c=0 """
    a, b, c = line
    x, y = pt
    return abs(a*x + b*y + c) / math.sqrt(a**2 + b**2)

# ==========================================
# PART 2: FEATURE MATCHING & GUI
# ==========================================

In [8]:
def match_feature_along_line(img1, img2, pt_clicked, F, window_size=15):
    """
    Searches for the corresponding point in img2 along the epipolar line.
    Uses Sum of Squared Differences (SSD).
    Reference: "Correspondence Problem", Slides 41-45.
    """
    h, w, _ = img2.shape
    
    # 1. Get Epipolar Line in Image 2
    line = compute_epipolar_line(pt_clicked, F, 1) # a*x + b*y + c = 0
    a, b, c = line
    
    # 2. Extract template from Image 1
    x, y = pt_clicked
    half_w = window_size // 2
    
    # Check bounds
    if x < half_w or x >= w - half_w or y < half_w or y >= h - half_w:
        print("Point too close to border.")
        return None

    template = img1[y-half_w:y+half_w+1, x-half_w:x+half_w+1].astype(np.float32)
    
    # 3. Scan along the epipolar line in Image 2
    best_score = float('inf')
    best_pt = None
    
    # We iterate x from 0 to width (assuming line isn't vertical)
    # Ideally we should trace the line pixels properly (Bresenham or similar)
    # Simple approximation: calculate y for every x
    
    for x2 in range(half_w, w - half_w):
        # Calculate y on the line: y = (-c - ax) / b
        if abs(b) > 1e-5:
            y2 = int((-c - a * x2) / b)
        else:
            continue # Vertical line handling skipped for brevity
            
        if y2 < half_w or y2 >= h - half_w:
            continue
            
        # Extract window in Image 2
        patch = img2[y2-half_w:y2+half_w+1, x2-half_w:x2+half_w+1].astype(np.float32)
        
        # Compute SSD (Sum of Squared Differences)
        score = np.sum((template - patch)**2)
        
        if score < best_score:
            best_score = score
            best_pt = (x2, y2)
            
    return best_pt

In [9]:
def mouse_callback(event, x, y, flags, param):
    global ref_points_1, ref_points_2, test_mode, F_matrix, img1, img2_display
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if not test_mode:
            # Calibration Phase: Collect pairs
            if len(ref_points_1) == len(ref_points_2):
                ref_points_1.append((x, y))
                print(f"Point {len(ref_points_1)} on Left Image recorded: ({x},{y}). Click corresponding point on Right.")
                cv2.circle(img_combined, (x, y), 5, (0, 0, 255), -1)
            else:
                # Adjust x for the right image (displayed side-by-side)
                real_x = x - img1.shape[1] 
                if real_x < 0:
                    print("Please click on the Right image.")
                    return
                ref_points_2.append((real_x, y))
                print(f"Point {len(ref_points_2)} on Right Image recorded: ({real_x},{y}).")
                cv2.circle(img_combined, (x, y), 5, (0, 255, 0), -1)
                
                if len(ref_points_1) == 8:
                    print("--- 8 Points collected. Press 'c' to Compute F or click more for better accuracy ---")

            cv2.imshow("Stereo Lab", img_combined)
            
        else:
            # Testing Phase: Feature Matching
            if x < img1.shape[1]: # Clicked on Left Image
                print(f"\nSearching match for ({x}, {y})...")
                
                # Draw point on left
                img_show = img_combined.copy()
                cv2.circle(img_show, (x, y), 5, (255, 0, 0), -1)
                
                # Find match
                match_pt = match_feature_along_line(img1, img2, (x,y), F_matrix)
                
                if match_pt:
                    print(f"Match found at: {match_pt}")
                    
                    # Draw Epipolar Line on Right Image
                    # line: ax + by + c = 0
                    line = compute_epipolar_line((x,y), F_matrix, 1)
                    x0, y0 = 0, int(-line[2]/line[1])
                    x1, y1 = img2.shape[1], int(-(line[2] + line[0]*img2.shape[1])/line[1])
                    
                    # Offset for visualization (right image starts at w)
                    w = img1.shape[1]
                    cv2.line(img_show, (x0 + w, y0), (x1 + w, y1), (0, 255, 255), 1)
                    
                    # Draw Crosshair on match
                    mx, my = match_pt
                    mx += w # Offset
                    cv2.drawMarker(img_show, (mx, my), (0, 255, 0), cv2.MARKER_CROSS, 20, 2)
                    
                    cv2.imshow("Stereo Lab", img_show)
                else:
                    print("Match not found (out of bounds).")

# ==========================================
# MAIN
# ==========================================

In [10]:

global img1, img2, img_combined, test_mode, F_matrix

# Load Images
print("Loading pic410.png and pic430.jpg...")
img1 = cv2.imread('./pic410.png')
img2 = cv2.imread('./pic430.png')


Loading pic410.png and pic430.jpg...


In [11]:
# Resize for easier viewing if too large
img1 = cv2.resize(img1, (0,0), fx=0.5, fy=0.5)
img2 = cv2.resize(img2, (0,0), fx=0.5, fy=0.5)

# Create side-by-side view
img_combined = np.hstack((img1, img2))

In [ ]:
cv2.namedWindow("Stereo Lab")
cv2.setMouseCallback("Stereo Lab", mouse_callback)

print("=======================================================")
print("PART 1: Manual Calibration")
print("1. Click a point on the LEFT image.")
print("2. Click the corresponding point on the RIGHT image.")
print("3. Repeat at least 8 times (10-12 recommended).")
print("4. Press 'c' to calculate Fundamental Matrix.")
print("=======================================================")


while True:
    cv2.imshow("Stereo Lab", img_combined)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
        
    elif key == ord('c') and len(ref_points_1) >= 8:
        # Separate Control Points (first 8) and Test Points (rest)
        ctrl_p1 = ref_points_1[:8]
        ctrl_p2 = ref_points_2[:8]
        
        test_p1 = ref_points_1[8:]
        test_p2 = ref_points_2[8:]
        
        print(f"\nComputing F using {len(ctrl_p1)} control points...")
        F_matrix = compute_fundamental_matrix(ctrl_p1, ctrl_p2)
        
        print("\nFundamental Matrix F:")
        print(F_matrix)
        
        e1, e2 = compute_epipoles(F_matrix)
        print(f"\nEpipole Left: {e1}")
        print(f"Epipole Right: {e2}")
        
        # Check Accuracy
        total_err = 0
        if len(test_p1) > 0:
            print("\nAccuracy Check (Distance to Epipolar Line):")
            for i in range(len(test_p1)):
                # Line in right image for point in left
                l2 = compute_epipolar_line(test_p1[i], F_matrix, 1)
                dist = calculate_distance_to_line(test_p2[i], l2)
                total_err += dist
                print(f"Test Point {i+1}: Distance = {dist:.4f} pixels")
            print(f"Average Error: {total_err/len(test_p1):.4f} pixels")
        else:
            print("\nNo extra points clicked for testing. (Click more than 8 next time!)")
            
        print("\n=======================================================")
        print("PART 2: Feature Matching")
        print("Click anywhere on the LEFT image to find its match in the RIGHT.")
        print("The search is constrained along the epipolar line.")
        print("Press 'q' to quit.")
        print("=======================================================")
        test_mode = True
        
cv2.destroyAllWindows()

PART 1: Manual Calibration
1. Click a point on the LEFT image.
2. Click the corresponding point on the RIGHT image.
3. Repeat at least 8 times (10-12 recommended).
4. Press 'c' to calculate Fundamental Matrix.
Point 1 on Right Image recorded: (106,207).
Point 2 on Left Image recorded: (151,191). Click corresponding point on Right.
Point 2 on Right Image recorded: (109,203).
Point 3 on Left Image recorded: (188,249). Click corresponding point on Right.
Point 3 on Right Image recorded: (149,271).
Point 4 on Left Image recorded: (52,153). Click corresponding point on Right.
Point 4 on Right Image recorded: (7,170).
Point 5 on Left Image recorded: (283,216). Click corresponding point on Right.
Point 5 on Right Image recorded: (151,201).
Point 6 on Left Image recorded: (385,248). Click corresponding point on Right.
Point 6 on Right Image recorded: (148,280).
Point 7 on Left Image recorded: (387,342). Click corresponding point on Right.
Point 7 on Right Image recorded: (15,297).
Point 8 on L

KeyboardInterrupt: 

: 